### 시작! : 전처리를 해보자

In [1]:
import requests
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.text import Tokenizer
import time
import numpy as np
import operator
from konlpy.tag import Komoran
from urllib.parse import quote_plus
import lxml.html
import pickle
import re
import keras
import pandas as pd

C:\Users\Admin\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [99]:
## pickle load 하기
with open('win_apply.txt', 'rb') as f:
    win_apply = pickle.load(f)

In [100]:
win_apply[0:2]

["‘무임승차하지 않는 삶’ 중학 시절, 은사님께서 해주신 말씀입니다. 직접 노력해서 결과를 얻어야 한다는 기본적 원칙은 곧 제 인생의 가치관이 되었습니다. 인생의 가치관이 생기자 새로운 목표가 생겼습니다. 그 계기는 고등학생 시절 학창시절 우연히 하게 된 일용직 근로자 경험에서 시작되었습니다. 작은 벽돌이 쌓여서 큰 건물이 완성되는 모습은 저에겐 큰 놀라움이었습니다. 당시 어린 눈으로 봤던 건설현장은 저의 가치관이 그대로 투영된 곳으로 다가왔고 곧, 건설현장은 저에게 목표가 되었습니다. 저는 건축엔지니어로서의 기본을 다지기 위해 아래와 같은 노력을 했습니다. 다양한 분야의 지식을 습득했습니다. 건축은 구조, 시공, 안전 등 다양한 분야로 이루어져 있습니다. 그 때문에 한 가지 분야만 능통해서는 온전한 건물을 지을 수 없습니다. 그래서 저는 설계, 안전, 환경, 시공 등 다양한 전공수업을 이수하였습니다. 다양한 변수가 있는 현장에서 습득한 지식을 토대로 공기단축에 이바지하도록 하겠습니다. 다양한 경험을 위해 경진대회에 참여했습니다. 학부 시절 시공경진 대회에 참여했습니다. 대회를 준비하는 과정에서의 공부는 시공과 하자에 대한 다양한 지식을 얻을 수 있었습니다. 또한, 21개 팀이 참가한 경진대회에서 100명이 넘는 인원과 충남지역 대기업 실무자분들 앞에서 발표한 경험은 저를 더욱 단단하게 만들어 주었고, '대상'이라는 성과도 얻을 수 있었습니다.",
 '가장 기억에 남는 전공은 안전공학입니다. 현직에 종사하시는 교수님 아래에서 안전의 이론과 더불어 현장에서의 적용을 동시에 배울 수 있었습니다. 안전교육법, 감성접근법 등을 배움은 분명 저의 강점이 될 것입니다. 이 수업을 통해 안전은 사람의 생명뿐만 아니라 공사의 성패와도 관련 있음을 알게 되었습니다. 안전이 현장에서 ‘필수’임을 깨달은 저는 건설안전기사를 취득하였습니다. 무재해 공사를 만들어 업계순위 10위 도약, 수주 10조 달성 등 비전 2020 달성에 앞장서겠습니다. 교양 과목에서 가장 관심 있었던 과목

In [7]:
# sentence 만들기
sentence = []
for i in range(0, len(win_apply)):
    a = win_apply[i].split(".")
    for j in range(0, len(a)):
        b = a[j]
        sentence.append(b)

In [8]:
## 명사추출
tagger = Komoran()

def get_noun(text):
    nouns = tagger.nouns(text)
    return [n for n in nouns if len(n) >= 2]  # 명사만 추출

words = []
for i in range(0, len(sentence)):
    a = get_noun(sentence[i])
    words.append(a)

In [9]:
print("문장수 :",len(sentence))
print("명사 추출 단어 수 :",len(words))

문장수 : 208550
명사 추출 단어 수 : 208550


In [10]:
# 전처리
none_index = []
for i in range(0, len(words)):
    if len(words[i]) == 0:
        none_index.append(i)

none_index.reverse()
        
for j in none_index:
    del sentence[j]
    del words[j]

print("값이 없는 리시트 삭제 후")
print("문장수 :",len(sentence))
print("명사 추출 단어 수 :",len(words))

max_index = []
for k in range(0, len(words)):
    if len(words[k]) > 42:
        max_index.append(k)

max_index.reverse()

for l in max_index:
    del words[l]
    del sentence[l]

print("\n")
print("명사 과다 추출된 리스트 삭제 후")
print("문장수 :",len(sentence))
print("명사 추출 단어 수 :",len(words))

값이 없는 리시트 삭제 후
문장수 : 187492
명사 추출 단어 수 : 187492


명사 과다 추출된 리스트 삭제 후
문장수 : 187400
명사 추출 단어 수 : 187400


### WORD - 임베딩 벡터를 만들어보자

In [11]:
# embedding
embedding_model = Word2Vec(words,
                           size=200,  # 100차원의 벡터로 바꾼다
                           window = 2,  # 주변 단어 앞 뒤로 두개까지
                           min_count = 1,  # 출현 빈도의 최소값, 나머진 삭제
                           workers = 4,  # CPU는 쿼드코어를 사용
                           iter = 100, # 100번 학습
                           sg = 1 # 분석방법론은 CBOW와 Skip-Gram 중 후자 선택
                          )
print(embedding_model)

Word2Vec(vocab=23845, size=200, alpha=0.025)


In [13]:
# 결과 확인
print(embedding_model.most_similar(positive=["장학금"],
                                   topn=10) # 단어와 가장 비슷한(유사도가 큰) 100개 단어 출력
     )

[('전액', 0.4947859048843384), ('문중', 0.4554343521595001), ('필진', 0.4424971342086792), ('차석', 0.430867075920105), ('주립 대학', 0.4147442877292633), ('국가장학금', 0.41453737020492554), ('기독교 선교단체', 0.41203415393829346), ('상패', 0.41156190633773804), ('단원고등학교', 0.4099908769130707), ('토플', 0.4084552824497223)]


/opt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


### Image 벡터 처럼 키워드를 임베딩 벡터로 바꾸어 보장

In [14]:
# words에서 키워드 하나만 살리기
for i in range(0, len(words)):
    words[i] = words[i][0]

In [15]:
words[0:5]

['무임승차', '노력', '인생', '계기', '벽돌']

In [11]:
# pickle에 저장하기
# with open('words_1_dic.txt', 'wb') as f:
#     pickle.dump(words, f)
# f.close

<function BufferedWriter.close>

In [7]:
# 데이터 로드하기
with open('words.txt', 'rb') as f:
    words= pickle.load(f)
f.close

<function BufferedReader.close>

In [16]:
# 워드 벡터 만들기
for i in range(0, len(words)):
    words[i] = embedding_model[words[i]]

/opt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# pickle에 저장하기
# with open('words_vec_200.txt', 'wb') as f:
#     pickle.dump(words, f)
# f.close

<function BufferedWriter.close>

In [18]:
with open('words_vec_200.txt', 'rb') as f:
    words = pickle.load(f)
f.close

<function BufferedReader.close>

In [19]:
with open('sentence.txt', 'wb') as f:
    pickle.dump(sentence, f)
f.close

<function BufferedWriter.close>

In [50]:
# 완성~
print(len(words))
print(len(sentence))

187400
187400


### 문단 전처리를 합시다!

In [12]:
## pickle load 하기
with open('win_apply.txt', 'rb') as f:
    para = pickle.load(f)
    
# 문단에 대해 명사를 추출
tagger = Komoran()

def get_noun(text):
    nouns = tagger.nouns(text)
    return [n for n in nouns if len(n) >= 2]  # 명사만 추출

words = []
for i in range(0, len(para)):
    a = get_noun(para[i])
    words.append(a)

 # words 추출된 단어 개수 확인
words_count = []
for i in range(0,len(words)):
    words_count.append(len(words[i]))

# 인덱스 화
words_keyword_dic = {w: i for i, w in enumerate(words_count)}

In [13]:
print(len(words))
print(len(para))

18605
18605


In [ ]:
"""
1. 300자 이하인 자소서를 제거
2. 모든 명사를 추출 및 확인
3. word2vec을 통해서 벡터화
4. padding을 통해서 맞춰주기 → 함수 만들기
5. 모델 돌리기
"""

In [20]:
"""
## 임베딩 벡터 만들기 ## 
데이터 전처리
words : 명사 추출 데이터
para : 문항 데이터
para_length : 문항의 최소 글자 수
"""

para_length = 400

index_over = []
for i in range(0, len(para)):
    if len(para[i]) < para_length:
        index_over.append(i)
    
index_over.reverse() # 역순으로
    
# index_over 값을 데이터에서 제거
for i in index_over:
    del words[i], para[i]
    
# padding을 위해서(나머지 자리 0으로 채워주기 위해서) 단어의 최고 자리 구하기
# words 추출된 단어 개수 확인
words_count = []
for i in range(0,len(words)):
    words_count.append(len(words[i]))
    
words_length_max = max(words_count) # max_length 설정
    
# Words Word2Vec
embedding_model = Word2Vec(words,
                           size = 100,  # 200차원의 벡터로 바꾼다
                           window = 2,  # 주변 단어 앞 뒤로 두개까지
                           min_count = 1,  # 출현 빈도의 최소값, 나머진 삭제
                           workers = 4,  # CPU는 쿼드코어를 사용
                           iter = 100, # 100번 학습
                           sg = 1 # 분석방법론은 CBOW와 Skip-Gram 중 후자 선택
                          )    


# 단어를 벡터 값으로 바꾸기
words_vec = []
for i in range(0, len(words)):
    words_vec_before = []
    
    for j in range(0, len(words[i])):
        a = embedding_model[words[i][j]]
        for k in range(0, len(a)):
            words_vec_before.append(a[k])
                
    words_vec.append(words_vec_before)

/opt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [15]:
print(len(words_vec))
print(len(words_vec[0]))

14918
19200


In [22]:
"""
dense vector shape fittning
벡터의 shape 맞추기! = padding : 0으로 채우자
"""
words_vec_length_count = []
for i in range(0, len(words_vec)):
    words_vec_length_count.append(len(words_vec[i]))

vec_max_length = max(words_vec_length_count) # 

for i in range(0, len(words_vec)):
    if len(words_vec[i]) != vec_max_length:
        for j in range(0, (vec_max_length-len(words_vec[i]))):
            words_vec[i].append(0)

In [23]:
len(words_vec[0])

56200

In [ ]:
words_vec = np.asarray(words_vec)

In [24]:
# pickle에 저장하기
# test0515 : 5차원 테스트용
# test0516 : 100차원 발표 결과용
with open('test0516.txt', 'wb') as f:
    pickle.dump(words_vec, f)
f.close

<function BufferedWriter.close>

In [25]:
# pickle에 저장하기
# para0515 : 5차원 테스트
# para0516 : 100차원 발표 결과용
with open('para0516.txt', 'wb') as f:
    pickle.dump(para, f)
f.close

<function BufferedWriter.close>

## 0523

In [3]:
## pickle load 하기
with open('Keyword0523.txt', 'rb') as f:
    words = pickle.load(f)
f.close()
    
## pickle load 하기
with open('para0523.txt', 'rb') as f:
    para = pickle.load(f)
f.close()

In [7]:
print(words[0:1])
print(para[0:1])

[['시공', '대회', '가치관', '건물', '건축', '투영', '경진', '중학', '경지', '건설', '일용직', '놀라움', '충남', '시절', '은사', '무임승차', '현장', '벽돌', '인생', '하자']]
["‘무임승차하지 않는 삶’ 중학 시절, 은사님께서 해주신 말씀입니다. 직접 노력해서 결과를 얻어야 한다는 기본적 원칙은 곧 제 인생의 가치관이 되었습니다. 인생의 가치관이 생기자 새로운 목표가 생겼습니다. 그 계기는 고등학생 시절 학창시절 우연히 하게 된 일용직 근로자 경험에서 시작되었습니다. 작은 벽돌이 쌓여서 큰 건물이 완성되는 모습은 저에겐 큰 놀라움이었습니다. 당시 어린 눈으로 봤던 건설현장은 저의 가치관이 그대로 투영된 곳으로 다가왔고 곧, 건설현장은 저에게 목표가 되었습니다. 저는 건축엔지니어로서의 기본을 다지기 위해 아래와 같은 노력을 했습니다. 다양한 분야의 지식을 습득했습니다. 건축은 구조, 시공, 안전 등 다양한 분야로 이루어져 있습니다. 그 때문에 한 가지 분야만 능통해서는 온전한 건물을 지을 수 없습니다. 그래서 저는 설계, 안전, 환경, 시공 등 다양한 전공수업을 이수하였습니다. 다양한 변수가 있는 현장에서 습득한 지식을 토대로 공기단축에 이바지하도록 하겠습니다. 다양한 경험을 위해 경진대회에 참여했습니다. 학부 시절 시공경진 대회에 참여했습니다. 대회를 준비하는 과정에서의 공부는 시공과 하자에 대한 다양한 지식을 얻을 수 있었습니다. 또한, 21개 팀이 참가한 경진대회에서 100명이 넘는 인원과 충남지역 대기업 실무자분들 앞에서 발표한 경험은 저를 더욱 단단하게 만들어 주었고, '대상'이라는 성과도 얻을 수 있었습니다."]


In [ ]:
"""
1. 추출 단어가 30개 미만, 200개 초과인거 삭제
2. 모든 명사를 추출 및 확인
3. word2vec을 통해서 벡터화 200차원
4. padding을 통해서 맞춰주기 → 함수 만들기
5. 모델 돌리기
"""

In [8]:
"""
## 임베딩 벡터 만들기 ## 
데이터 전처리
words : 명사 추출 데이터
para : 문항 데이터
para_length : 문항의 최소 글자 수
"""
"""
[0523 keyword, paragraph 파일]
추출 단어가 30개 미만, 200개 초과인거 삭제
"""

"""
이 부분은 이미 진행되었으므로 skip
para_length = 400

index_over = []
for i in range(0, len(para)):
    if len(para[i]) < para_length:
        index_over.append(i)
    
index_over.reverse() # 역순으로
    
# index_over 값을 데이터에서 제거
for i in index_over:
    del words[i], para[i]

"""    
# padding을 위해서(나머지 자리 0으로 채워주기 위해서) 단어의 최고 자리 구하기
# words 추출된 단어 개수 확인
words_count = []
for i in range(0,len(words)):
    words_count.append(len(words[i]))
    
words_length_max = max(words_count) # max_length 설정
    
# Words Word2Vec
embedding_model = Word2Vec(words,
                           size = 200,  # 200차원의 벡터로 바꾼다
                           window = 2,  # 주변 단어 앞 뒤로 두개까지
                           min_count = 1,  # 출현 빈도의 최소값, 나머진 삭제
                           workers = 4,  # CPU는 쿼드코어를 사용
                           iter = 100, # 100번 학습
                           sg = 1 # 분석방법론은 CBOW와 Skip-Gram 중 후자 선택
                          )    


# 단어를 벡터 값으로 바꾸기
words_vec = []
for i in range(0, len(words)):
    words_vec_before = []
    
    for j in range(0, len(words[i])):
        a = embedding_model[words[i][j]]
        for k in range(0, len(a)):
            words_vec_before.append(a[k])
                
    words_vec.append(words_vec_before)

/opt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:54: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [9]:
print(len(words_vec))
print(len(words_vec[0]))

17090
4000


In [10]:
"""
dense vector shape fittning
벡터의 shape 맞추기! = padding : 0으로 채우자
"""
words_vec_length_count = []
for i in range(0, len(words_vec)):
    words_vec_length_count.append(len(words_vec[i]))

vec_max_length = max(words_vec_length_count) # 

for i in range(0, len(words_vec)):
    if len(words_vec[i]) != vec_max_length:
        for j in range(0, (vec_max_length-len(words_vec[i]))):
            words_vec[i].append(0)

In [11]:
len(words_vec[0])

4000

In [12]:
words_vec = np.asarray(words_vec)

In [13]:
# pickle에 저장하기
# test0515 : 5차원 테스트용
# test0516 : 100차원 발표 결과용
# test0523 : 200차원 면담 결과용
with open('test0523.txt', 'wb') as f:
    pickle.dump(words_vec, f)

### 영어 자소서 확인

In [93]:
import pickle
import numpy as np

In [94]:
with open('para0523.txt', 'rb') as f:
    para = np.array(pickle.load(f))
        
with open('test0523.txt', 'rb') as f:
    test = np.array(pickle.load(f))

In [95]:
index = []
for i in range(0, len(para)):
    if (("a" in para[i]) & ("b" in para[i]) & ("c" in para[i]) & ("d" in para[i]) & ("e" in para[i]) & ("f" in para[i]) & ("g" in para[i]) & ("h" in para[i]) & ("i" in para[i]) ):
        index.append(i)

print("인덱스 개수 : ",len(index))
print("변경 전 : ", len(para))
print("변경 전 : ",len(test))

# list로 바꿔서 해당 인덱스 삭제
para = para.tolist()
test = test.tolist()
index.reverse()
for i in index:
    del para[i]
    del test[i]
    
print("변경 후 : ", len(para))
print("변경 후 : ",len(test))

인덱스 개수 :  42
변경 전 :  17090
변경 전 :  17090
변경 후 :  17048
변경 후 :  17048


In [97]:
with open('test0528.txt', 'wb') as f:
    pickle.dump(test, f)
    
with open('para0528.txt', 'wb') as f:
    pickle.dump(para, f)